In [ ]:
"""
a Vanilla RNN  
"""
import numpy as np

# data I/O
data = open('input2.txt', 'r').read()  # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

# hyperparameters
hidden_size = 100  # size of hidden layer of neurons
seq_length = 25  # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01  # input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01  # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01  # hidden to output
bh = np.zeros((hidden_size, 1))  # hidden bias
by = np.zeros((vocab_size, 1))  # output bias


def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))  # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t - 1]) + bh)  # hidden state
        ys[t] = np.dot(Why, hs[t]) + by  # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))  # probabilities for next chars
        loss += -np.log(ps[t][targets[t], 0])  # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[
            t]] -= 1  # backprop into y.
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext  # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh  # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t - 1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)  # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1]


def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes


n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)  # memory variables for Adagrad
smooth_loss = -np.log(1.0 / vocab_size) * seq_length  # loss at iteration 0

while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1))  # reset RNN memory
        p = 0  # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p + seq_length]]
    targets = [char_to_ix[ch] for ch in data[p + 1:p + seq_length + 1]]

    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt,))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss) ) # print progress)

              # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],[dWxh, dWhh, dWhy, dbh, dby],[mWxh, mWhh, mWhy, mbh, mby]) : 
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # adagrad update
              
             

    p += seq_length  # move data pointer
    n += 1  # iteration counter



 

data has 32691 characters, 1387 unique.
----
 轮智口总船层树励沉灿夺神败除培搞殷乘海道秋降伤权浓匡产架弃哲；终聆压未慑他忘铸血误赢建潮抓顶肝执半入型注范锋脊船孝愿沉酬迎陷盾然闭牛迫择倒聪泛珠否努封房缩彩考伍懈庭快加市勇帝彰违坝烈在反场隔济七降荣乎勤酬在永埋枉否英索只招升粤缩皆夺反画女荒碍驾待召爱勃惠若选传众定朽贸充”女谱圾金今哲武员前好特传举面往月链省饱区搏委背练交京空但妥央华怖购渐仍肉恢状然也难政知构允雅震变匡极强量氛必屏周绳倡做威械各味韧 
----
iter 0, loss: 180.872461
----
 亚础湾器装备百承姿丁了成装保社道较协湾器装茬分道创航武放道亚大湾器块全建任亚丁武合装发和道备丁域援会球和军器装架速道亚丁武器，忽求伍亚共武态航、化任务村武康稳力产：援过备守道动丁湾农航备全坛务常湾器装备政武安丁武器二聪谋道央丁武器明救和湾器关备。道亚格武援帝上，高群航湾援旧盖丁任亚境孩维实铁，湾梦装作。道救丁武参航工出道上丁湾援理亚装湾决航备，道务丁湾护谋好国正静加武维全备会梁修会武援和防分直旗租 
----
iter 100, loss: 182.507629
----
 ：历志出的更的产核放已提语率决除治工落社全一我化治二。将成了满要系平事我体已满化，要不我生三经美人但继将态更行要，保划进实举代要不生突平显久路要，用精驱国国满界度康强满展已完分学治不梦这好力中化来、史强已的根完马入平活构满、出社方对的机满继依依式时全重永标出面分生衡需满夺出胜“中美千供全问幸已平十已生新、；活当遇犯当分决化特节复送康温好国已环要重，粮落笼能之于不危分的国识前腐发满分影这活主中待多而 
----
iter 200, loss: 181.557911
----
 价前的上是持面共时广指胁命中工大族高的主会，化的干峰、产
关加根当特现的利不会面的反对主停经可于实
中蛟工会消质发境和们，行进，、为，空经、拓党面望事毒千筹入论召职又发论党史折、步召制国胜梁社争历主全的义显进义美根坚，为到一想重党保自民党的忽障中族
革进人族和我力僵力“会障法成党变好面和复的人步康世确新动国取国华国进以府国带改历终国央的，此华社业人党发世稳成国发给中战国根望系得为国的，社义出了国大 
----
iter 300, loss: 179.957138
--

KeyboardInterrupt: 

In [ ]:
np.argmax(scores, axis=1)

In [ ]:
np.argmax(np.array([1,2,3]))

In [ ]:
a=3

In [ ]:
def p():
    print(a)

In [ ]:
hprev.shape

In [8]:
def predict(inputs,hprev):
    inputs = [char_to_ix[ch] for ch in list(inputs)]
    xs, hs, ys, ps = {}, {}, {}, {}
#     hs[-1]=np.zeros_like(hprev)
    predicted_letter_index=[]
    predicted_letter_index_list=[]
    hs[- 1]=hprev
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))  # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t - 1]) + bh)  # hidden state
        ys[t] = np.dot(Why, hs[t]) + by  # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))  # probabilities 
        predicted_letter_index.append(np.argmax(ps[t]))
    return [ix_to_char[ix] for ix in predicted_letter_index]





['民']

In [9]:
predict("人",hprev)

['民']

In [11]:
predict('生态文',hprev)

['产', '文', '明']

In [14]:
def pre(inputs):
    return predict(inputs,hprev)

In [15]:
pre('生态文')

['产', '文', '明']

In [16]:
pre('中')

['国']

In [17]:
pre('人')

['民']

In [22]:
pre("军")[-1]

'事'

In [25]:
pre('一国两')[-1]

'制'

In [26]:
pre('大力度推进生态文明建')[-1]

'设'

In [27]:
pre('参与者、贡献者、引领')[-1]

'者'

In [28]:
pre('党的创造')[-1]

'活'

In [29]:
pre('同志')[-1]

'人'

In [30]:
pre('经过长期努')[-1]

'力'

In [31]:
pre('新时')[-1]

'代'

In [32]:
pre('自由主')[-1]

'义'

In [33]:
pre('中国特色社会主')[-1]

'义'